In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from collections import deque
import pickle
import time

In [2]:
file_path = os.path.join("Complete saved rows", 'saved_row_284.pkl')
with open(file_path,'rb') as f:
    examine = pickle.load(f)

In [3]:
np.array(examine[11])

array(['Upminster', 'Upminster Bridge', 'Hornchurch', 'Elm Park',
       'Dagenham East', 'Dagenham Heathway', 'Becontree', 'Upney',
       'Barking', 'Woodgrange Park', 'Wanstead Park',
       'Leytonstone High Road', 'Leyton Midland Road',
       "Walthamstow Queen's Road", 'Blackhorse Road', 'Tottenham Hale',
       'Seven Sisters', 'Finsbury Park'], dtype='<U24')

In [2]:
entry_exit_data = pd.read_excel('Expanded Station Entry Exit Data2.xls')
entry_exit_data

,Station,Borough,Entry Weekday,Entry Saturday,Entry Sunday,Exit Weekday,Exit Saturday,Exit Sunday,Total,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,Upney,Barking & Dagenham,4622,2785,1813,4182,2372,1619,2.567807e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dagenham East,Barking & Dagenham,5082,3369,2099,4593,2770,1795,2.850852e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Becontree,Barking & Dagenham,6034,3935,2549,5638,3595,2423,3.458723e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dagenham Heathway,Barking & Dagenham,9800,6422,4174,9711,6157,4109,5.779848e+06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barking,Barking & Dagenham,29410,19930,14113,28165,18712,13372,1.727858e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,Paddington,"Westminster,City of London",78992,48872,41733,76087,46671,36560,4.633128e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
329,Bank & Monument,"Westminster,City of London",112547,29906,17414,111709,29100,16333,5.925607e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330,Liverpool Street,"Westminster,City of London",132900,66607,53289,133666,70739,50597,7.692981e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331,Victoria,"Westminster,City of London",121018,88605,69760,122783,91001,71565,7.516316e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
tubemap = {
"Acton Central" : ["Willesden Junction","South Acton"],
"Acton Town" : ["Ealing Common","South Ealing","Turnham Green","Chiswick Park"],
"Aldgate" : ["Liverpool Street","Tower Hill"],
"Aldgate East" : ["Tower Hill","Whitechapel","Liverpool Street"],
"All Saints" : ["Langdon Park","Poplar"],
"Alperton" : ["Park Royal","Sudbury Town"],
"Anerley" : ["Penge West","Norwood Junction"],
"Angel" : ["King's Cross St. Pancras","Old Street"],
"Archway" : ["Highgate","Tufnell Park"],
"Arnos Grove" : ["Bounds Green","Southgate"],
"Arsenal" : ["Finsbury Park","Holloway Road"],
"Baker Street" : ["Marylebone","Regent's Park","Edgware Road (C)","Great Portland Street","Bond Street","St. John's Wood","Finchley Road"],
"Balham" : ["Clapham South","Tooting Bec"],
"Bank & Monument" : ["Liverpool Street","St. Paul's","Shadwell","London Bridge","Moorgate","Waterloo","Tower Hill","Cannon Street"],
"Barbican" : ["Farringdon","Moorgate"],
"Barking" : ["East Ham","Woodgrange Park","Upney"],
"Barkingside" : ["Fairlop","Newbury Park"],
"Barons Court" : ["Hammersmith","West Kensington","Earl's Court"],
"Bayswater" : ["Notting Hill Gate","Paddington"],
"Beckton" : ["Gallions Reach"],
"Beckton Park" : ["Cyprus","Royal Albert"],
"Becontree" : ["Dagenham Heathway","Upney"],
"Belsize Park" : ["Chalk Farm","Hampstead"],
"Bermondsey" : ["Canada Water","London Bridge"],
"Bethnal Green" : ["Liverpool Street","Mile End"],
"Blackfriars" : ["Mansion House","Temple"],
"Blackhorse Road" : ["Tottenham Hale","Walthamstow Central","South Tottenham","Walthamstow Queen's Road"],
"Blackwall" : ["East India","Poplar"],
"Bond Street" : ["Marble Arch","Oxford Circus","Green Park","Baker Street"],
"Borough" : ["Elephant & Castle","London Bridge"],
"Boston Manor" : ["Northfields","Osterley"],
"Bounds Green" : ["Wood Green","Arnos Grove"],
"Bow Road" : ["Bromley-by-Bow","Mile End","Devons Road","Pudding Mill Lane"],
"Brent Cross" : ["Golders Green","Hendon Central"],
"Brixton" : ["Stockwell"],
"Brockley" : ["New Cross Gate","Honor Oak Park"],
"Bromley-by-Bow" : ["West Ham","Bow Road"],
"Brondesbury" : ["Brondesbury Park","West Hampstead"],
"Brondesbury Park" : ["Kensal Rise","Brondesbury"],
"Burnt Oak" : ["Colindale","Edgware"],
"Caledonian Road" : ["Holloway Road","King's Cross St. Pancras"],
"Caledonian Road & Barnsbury" : ["Camden Road","Highbury & Islington"],
"Camden Road" : ["Kentish Town West","Caledonian Road & Barnsbury"],
"Camden Town" : ["Chalk Farm","Euston","Kentish Town","Mornington Crescent"],
"Canada Water" : ["Rotherhithe","Surrey Quays","Canary Wharf","Bermondsey"],
"Canary Wharf" : ["Heron Quays","West India Quay","North Greenwich","Canada Water"],
"Canning Town" : ["East India","Royal Victoria","North Greenwich","West Ham","West Silvertown"],
"Cannon Street" : ["Mansion House","Bank & Monument"],
"Canonbury" : ["Highbury & Islington","Dalston Kingsland"],
"Canons Park" : ["Queensbury","Stanmore"],
"Chalk Farm" : ["Belsize Park","Camden Town"],
"Chancery Lane" : ["Holborn","St. Paul's"],
"Charing Cross" : ["Embankment","Piccadilly Circus","Leicester Square"],
"Chigwell" : ["Grange Hill","Roding Valley"],
"Chiswick Park" : ["Acton Town","Turnham Green"],
"Clapham Common" : ["Clapham North","Clapham South"],
"Clapham Junction" : ["Imperial Wharf"],
"Clapham North" : ["Stockwell","Clapham Common"],
"Clapham South" : ["Balham","Clapham Common"],
"Cockfosters" : ["Oakwood"],
"Colindale" : ["Hendon Central","Burnt Oak"],
"Colliers Wood" : ["South Wimbledon","Tooting Broadway"],
"Covent Garden" : ["Holborn","Leicester Square"],
"Crossharbour" : ["Mudchute","South Quay"],
"Crouch Hill" : ["Upper Holloway","Harringay Green Lanes"],
"Croxley" : ["Moor Park"],
"Crystal Palace" : ["Sydenham"],
"Custom House for ExCel." : ["Prince Regent","Royal Victoria"],
"Cutty Sark for Maritime Greenwich" : ["Greenwich","Island Gardens"],
"Cyprus" : ["Gallions Reach","Beckton Park"],
"Dagenham East" : ["Dagenham Heathway","Elm Park"],
"Dagenham Heathway" : ["Becontree","Dagenham East"],
"Dalston Kingsland" : ["Haggerston","Canonbury","Hackney Central"],
"Deptford Bridge" : ["Elverson Road","Greenwich"],
"Devons Road" : ["Langdon Park","Bow Road"],
"Dollis Hill" : ["Neasden","Willesden Green"],
"Ealing Broadway" : ["West Acton","Ealing Common"],
"Ealing Common" : ["Acton Town","North Ealing","Ealing Broadway"],
"Earl's Court" : ["Gloucester Road","High Street Kensington","Kensington (Olympia)","West Brompton","West Kensington","Barons Court"],
"East Acton" : ["North Acton","White City"],
"East Finchley" : ["Finchley Central","Highgate"],
"East Ham" : ["Upton Park","Barking"],
"East India" : ["Blackwall","Canning Town"],
"East Putney" : ["Putney Bridge","Southfields"],
"Eastcote" : ["Rayners Lane","Ruislip Manor"],
"Edgware" : ["Burnt Oak"],
"Edgware Road (B)" : ["Marylebone","Paddington"],
"Edgware Road (C)" : ["Paddington","Baker Street"],
"Elephant & Castle" : ["Lambeth North","Kennington","Borough"],
"Elm Park" : ["Hornchurch","Dagenham East"],
"Elverson Road" : ["Lewisham","Deptford Bridge"],
"Embankment" : ["Waterloo","Temple","Westminster","Charing Cross"],
"Euston" : ["South Hampstead","King's Cross St. Pancras","Mornington Crescent","Warren Street","Camden Town"],
"Euston Square" : ["Great Portland Street","King's Cross St. Pancras"],
"Fairlop" : ["Hainault","Barkingside"],
"Farringdon" : ["King's Cross St. Pancras","Barbican"],
"Finchley Central" : ["Mill Hill East","West Finchley","East Finchley"],
"Finchley Road" : ["Swiss Cottage","West Hampstead","Wembley Park","Baker Street"],
"Finchley Road & Frognal" : ["West Hampstead","Hampstead Heath"],
"Finsbury Park" : ["Manor House","Highbury & Islington","Seven Sisters","Arsenal"],
"Forest Hill" : ["Honor Oak Park","Sydenham"],
"Fulham Broadway" : ["Parsons Green","West Brompton"],
"Gallions Reach" : ["Beckton","Cyprus"],
"Gants Hill" : ["Newbury Park","Redbridge"],
"Gloucester Road" : ["High Street Kensington","South Kensington","Earl's Court"],
"Golders Green" : ["Hampstead","Brent Cross"],
"Goldhawk Road" : ["Hammersmith","Shepherd's Bush Market"],
"Goodge Street" : ["Tottenham Court Road","Warren Street"],
"Gospel Oak" : ["Hampstead Heath","Kentish Town West","Upper Holloway"],
"Grange Hill" : ["Hainault","Chigwell"],
"Great Portland Street" : ["Baker Street","Euston Square"],
"Green Park" : ["Westminster","Hyde Park Corner","Piccadilly Circus","Oxford Circus","Victoria","Bond Street"],
"Greenford" : ["Northolt","Perivale"],
"Greenwich" : ["Cutty Sark for Maritime Greenwich","Deptford Bridge"],
"Gunnersbury" : ["South Acton","Kew Gardens","Turnham Green"],
"Hackney Central" : ["Dalston Kingsland","Homerton"],
"Hackney Wick" : ["Homerton","Stratford"],
"Haggerston" : ["Hoxton","Dalston Kingsland"],
"Hainault" : ["Fairlop","Grange Hill"],
"Hammersmith" : ["Ravenscourt Park","Turnham Green","Barons Court","Goldhawk Road"],
"Hampstead" : ["Belsize Park","Golders Green"],
"Hampstead Heath" : ["Finchley Road & Frognal","Gospel Oak"],
"Hanger Lane" : ["North Acton","Perivale"],
"Harlesden" : ["Stonebridge Park","Willesden Junction"],
"Harringay Green Lanes" : ["Crouch Hill","South Tottenham"],
"Harrow & Wealdstone" : ["Kenton","Headstone Lane"],
"Harrow-on-the-Hill" : ["Northwick Park","North Harrow","West Harrow"],
"Hatch End" : ["Headstone Lane"],
"Hatton Cross" : ["Hounslow West"],
"Headstone Lane" : ["Harrow & Wealdstone","Hatch End"],
"Hendon Central" : ["Brent Cross","Colindale"],
"Heron Quays" : ["South Quay","Canary Wharf"],
"High Barnet" : ["Totteridge & Whetstone"],
"High Street Kensington" : ["Notting Hill Gate","Earl's Court","Gloucester Road"],
"Highbury & Islington" : ["King's Cross St. Pancras","Finsbury Park","Caledonian Road & Barnsbury","Canonbury"],
"Highgate" : ["Archway","East Finchley"],
"Hillingdon" : ["Ickenham","Uxbridge"],
"Holborn" : ["Tottenham Court Road","Russell Square","Chancery Lane","Covent Garden"],
"Holland Park" : ["Notting Hill Gate","Shepherd's Bush"],
"Holloway Road" : ["Arsenal","Caledonian Road"],
"Homerton" : ["Hackney Central","Hackney Wick"],
"Honor Oak Park" : ["Brockley","Forest Hill"],
"Hornchurch" : ["Upminster Bridge","Elm Park"],
"Hounslow Central" : ["Hounslow East","Hounslow West"],
"Hounslow East" : ["Osterley","Hounslow Central"],
"Hounslow West" : ["Hatton Cross","Hounslow Central"],
"Hoxton" : ["Shoreditch High Street","Haggerston"],
"Hyde Park Corner" : ["Knightsbridge","Green Park"],
"Ickenham" : ["Ruislip","Hillingdon"],
"Imperial Wharf" : ["West Brompton","Clapham Junction"],
"Island Gardens" : ["Mudchute","Cutty Sark for Maritime Greenwich"],
"Kennington" : ["Oval","Waterloo","Elephant & Castle"],
"Kensal Green" : ["Queen's Park","Willesden Junction"],
"Kensal Rise" : ["Willesden Junction","Brondesbury Park"],
"Kensington (Olympia)" : ["Shepherd's Bush","Earl's Court","West Brompton"],
"Kentish Town" : ["Tufnell Park","Camden Town"],
"Kentish Town West" : ["Gospel Oak","Camden Road"],
"Kenton" : ["South Kenton","Harrow & Wealdstone"],
"Kew Gardens" : ["Richmond","Gunnersbury"],
"Kilburn" : ["West Hampstead","Willesden Green"],
"Kilburn High Road" : ["Queen's Park","South Hampstead"],
"Kilburn Park" : ["Maida Vale","Queen's Park"],
"King George V" : ["Pontoon Dock","Woolwich Arsenal"],
"King's Cross St. Pancras" : ["Russell Square","Angel","Caledonian Road","Euston","Euston Square","Farringdon","Highbury & Islington"],
"Kingsbury" : ["Queensbury","Wembley Park"],
"Knightsbridge" : ["South Kensington","Hyde Park Corner"],
"Ladbroke Grove" : ["Latimer Road","Westbourne Park"],
"Lambeth North" : ["Waterloo","Elephant & Castle"],
"Lancaster Gate" : ["Marble Arch","Queensway"],
"Langdon Park" : ["All Saints","Devons Road"],
"Latimer Road" : ["White City","Ladbroke Grove"],
"Leyton Midland Road" : ["Walthamstow Queen's Road","Leytonstone High Road"],
"Leytonstone High Road" : ["Leyton Midland Road","Wanstead Park"],
"Leicester Square" : ["Tottenham Court Road","Piccadilly Circus","Charing Cross","Covent Garden"],
"Lewisham" : ["Elverson Road"],
"Leyton" : ["Leytonstone","Stratford"],
"Leytonstone" : ["Snaresbrook","Wanstead","Leyton"],
"Limehouse" : ["Shadwell","Westferry"],
"Liverpool Street" : ["Moorgate","Aldgate","Aldgate East","Bank & Monument","Bethnal Green"],
"London Bridge" : ["Southwark","Bank & Monument","Bermondsey","Borough"],
"Maida Vale" : ["Warwick Avenue","Kilburn Park"],
"Manor House" : ["Turnpike Lane","Finsbury Park"],
"Mansion House" : ["Blackfriars","Cannon Street"],
"Marble Arch" : ["Bond Street","Lancaster Gate"],
"Marylebone" : ["Baker Street","Edgware Road (B)"],
"Mile End" : ["Stratford","Stepney Green","Bethnal Green","Bow Road"],
"Mill Hill East" : ["Finchley Central"],
"Moor Park" : ["Northwood","Croxley"],
"Moorgate" : ["Old Street","Bank & Monument","Barbican","Liverpool Street"],
"Morden" : ["South Wimbledon"],
"Mornington Crescent" : ["Camden Town","Euston"],
"Mudchute" : ["Crossharbour","Island Gardens"],
"Neasden" : ["Wembley Park","Dollis Hill"],
"New Cross" : ["Surrey Quays"],
"New Cross Gate" : ["Surrey Quays","Brockley"],
"Newbury Park" : ["Barkingside","Gants Hill"],
"North Acton" : ["West Acton","East Acton","Hanger Lane"],
"North Ealing" : ["Park Royal","Ealing Common"],
"North Greenwich" : ["Canary Wharf","Canning Town"],
"North Harrow" : ["Pinner","Harrow-on-the-Hill"],
"North Wembley" : ["South Kenton","Wembley Central"],
"Northfields" : ["South Ealing","Boston Manor"],
"Northolt" : ["South Ruislip","Greenford"],
"Northwick Park" : ["Preston Road","Harrow-on-the-Hill"],
"Northwood" : ["Northwood Hills","Moor Park"],
"Northwood Hills" : ["Pinner","Northwood"],
"Norwood Junction" : ["Anerley","West Croydon"],
"Notting Hill Gate" : ["Queensway","Bayswater","High Street Kensington","Holland Park"],
"Oakwood" : ["Southgate","Cockfosters"],
"Old Street" : ["Angel","Moorgate"],
"Osterley" : ["Boston Manor","Hounslow East"],
"Oval" : ["Stockwell","Kennington"],
"Oxford Circus" : ["Piccadilly Circus","Regent's Park","Tottenham Court Road","Warren Street","Bond Street","Green Park"],
"Paddington" : ["Warwick Avenue","Royal Oak","Bayswater","Edgware Road (B)","Edgware Road (C)"],
"Park Royal" : ["Alperton","North Ealing"],
"Parsons Green" : ["Putney Bridge","Fulham Broadway"],
"Penge West" : ["Sydenham","Anerley"],
"Perivale" : ["Greenford","Hanger Lane"],
"Piccadilly Circus" : ["Charing Cross","Green Park","Leicester Square","Oxford Circus"],
"Pimlico" : ["Vauxhall","Victoria"],
"Pinner" : ["North Harrow","Northwood Hills"],
"Plaistow" : ["Upton Park","West Ham"],
"Pontoon Dock" : ["West Silvertown","King George V"],
"Poplar" : ["Westferry","West India Quay","All Saints","Blackwall"],
"Preston Road" : ["Wembley Park","Northwick Park"],
"Prince Regent" : ["Royal Albert","Custom House for ExCel."],
"Pudding Mill Lane" : ["Stratford","Bow Road"],
"Putney Bridge" : ["East Putney","Parsons Green"],
"Queen's Park" : ["Kensal Green","Kilburn Park","Kilburn High Road"],
"Queensbury" : ["Canons Park","Kingsbury"],
"Queensway" : ["Lancaster Gate","Notting Hill Gate"],
"Ravenscourt Park" : ["Stamford Brook","Hammersmith"],
"Rayners Lane" : ["West Harrow","South Harrow","Eastcote"],
"Redbridge" : ["Wanstead","Gants Hill"],
"Regent's Park" : ["Baker Street","Oxford Circus"],
"Richmond" : ["Kew Gardens"],
"Roding Valley" : ["Woodford","Chigwell"],
"Rotherhithe" : ["Wapping","Canada Water"],
"Royal Albert" : ["Beckton Park","Prince Regent"],
"Royal Oak" : ["Westbourne Park","Paddington"],
"Royal Victoria" : ["Canning Town","Custom House for ExCel."],
"Ruislip" : ["Ruislip Manor","Ickenham"],
"Ruislip Gardens" : ["South Ruislip","West Ruislip"],
"Ruislip Manor" : ["Eastcote","Ruislip"],
"Russell Square" : ["Holborn","King's Cross St. Pancras"],
"Seven Sisters" : ["Tottenham Hale","Finsbury Park"],
"Shadwell" : ["Tower Hill","Wapping","Whitechapel","Bank & Monument","Limehouse"],
"Shepherd's Bush" : ["White City","Holland Park","Willesden Junction","Kensington (Olympia)"],
"Shepherd's Bush Market" : ["Goldhawk Road","White City"],
"Shoreditch High Street" : ["Whitechapel","Hoxton"],
"Sloane Square" : ["South Kensington","Victoria"],
"Snaresbrook" : ["South Woodford","Leytonstone"],
"South Acton" : ["Acton Central","Gunnersbury"],
"South Ealing" : ["Acton Town","Northfields"],
"South Hampstead" : ["Kilburn High Road","Euston"],
"South Harrow" : ["Sudbury Hill","Rayners Lane"],
"South Kensington" : ["Gloucester Road","Knightsbridge","Sloane Square"],
"South Kenton" : ["Kenton","North Wembley"],
"South Quay" : ["Crossharbour","Heron Quays"],
"South Ruislip" : ["Northolt","Ruislip Gardens"],
"South Tottenham" : ["Harringay Green Lanes","Blackhorse Road"],
"South Wimbledon" : ["Colliers Wood","Morden"],
"South Woodford" : ["Woodford","Snaresbrook"],
"Southfields" : ["Wimbledon Park","East Putney"],
"Southgate" : ["Arnos Grove","Oakwood"],
"Southwark" : ["Waterloo","London Bridge"],
"St. James's Park" : ["Victoria","Westminster"],
"St. John's Wood" : ["Swiss Cottage","Baker Street"],
"St. Paul's" : ["Bank & Monument","Chancery Lane"],
"Stamford Brook" : ["Turnham Green","Ravenscourt Park"],
"Stanmore" : ["Canons Park"],
"Stepney Green" : ["Whitechapel","Mile End"],
"Stockwell" : ["Vauxhall","Brixton","Clapham North","Oval"],
"Stonebridge Park" : ["Wembley Central","Harlesden"],
"Stratford" : ["West Ham","Leyton","Hackney Wick","Mile End","Pudding Mill Lane"],
"Sudbury Hill" : ["Sudbury Town","South Harrow"],
"Sudbury Town" : ["Alperton","Sudbury Hill"],
"Surrey Quays" : ["Canada Water","New Cross","New Cross Gate"],
"Swiss Cottage" : ["Finchley Road","St. John's Wood"],
"Sydenham" : ["Forest Hill","Crystal Palace","Penge West"],
"Temple" : ["Blackfriars","Embankment"],
"Tooting Bec" : ["Tooting Broadway","Balham"],
"Tooting Broadway" : ["Colliers Wood","Tooting Bec"],
"Tottenham Court Road" : ["Goodge Street","Holborn","Leicester Square","Oxford Circus"],
"Tottenham Hale" : ["Blackhorse Road","Seven Sisters"],
"Totteridge & Whetstone" : ["Woodside Park","High Barnet"],
"Tower Hill" : ["Aldgate","Aldgate East","Bank & Monument","Shadwell"],
"Tufnell Park" : ["Archway","Kentish Town"],
"Turnham Green" : ["Acton Town","Chiswick Park","Gunnersbury","Hammersmith","Stamford Brook"],
"Turnpike Lane" : ["Wood Green","Manor House"],
"Upminster" : ["Upminster Bridge"],
"Upminster Bridge" : ["Hornchurch","Upminster"],
"Upney" : ["Barking","Becontree"],
"Upper Holloway" : ["Gospel Oak","Crouch Hill"],
"Upton Park" : ["East Ham","Plaistow"],
"Uxbridge" : ["Hillingdon"],
"Vauxhall" : ["Pimlico","Stockwell"],
"Victoria" : ["Green Park","Pimlico","Sloane Square","St. James's Park"],
"Walthamstow Central" : ["Blackhorse Road"],
"Walthamstow Queen's Road" : ["Blackhorse Road","Leyton Midland Road"],
"Wanstead" : ["Leytonstone","Redbridge"],
"Wanstead Park" : ["Leytonstone High Road","Woodgrange Park"],
"Wapping" : ["Rotherhithe","Shadwell"],
"Warren Street" : ["Euston","Goodge Street","Oxford Circus"],
"Warwick Avenue" : ["Maida Vale","Paddington"],
"Waterloo" : ["Westminster","Bank & Monument","Embankment","Kennington","Lambeth North","Southwark"],
"Wembley Central" : ["North Wembley","Stonebridge Park"],
"Wembley Park" : ["Finchley Road","Kingsbury","Neasden","Preston Road"],
"West Acton" : ["Ealing Broadway","North Acton"],
"West Brompton" : ["Earl's Court","Fulham Broadway","Kensington (Olympia)","Imperial Wharf"],
"West Croydon" : ["Norwood Junction"],
"West Finchley" : ["Woodside Park","Finchley Central"],
"West Ham" : ["Bromley-by-Bow","Canning Town","Plaistow","Stratford"],
"West Hampstead" : ["Finchley Road","Kilburn","Brondesbury","Finchley Road & Frognal"],
"West Harrow" : ["Harrow-on-the-Hill","Rayners Lane"],
"West India Quay" : ["Canary Wharf","Poplar","Westferry"],
"West Kensington" : ["Barons Court","Earl's Court"],
"West Ruislip" : ["Ruislip Gardens"],
"West Silvertown" : ["Canning Town","Pontoon Dock"],
"Westbourne Park" : ["Ladbroke Grove","Royal Oak"],
"Westferry" : ["West India Quay","Limehouse","Poplar"],
"Westminster" : ["Embankment","Green Park","St. James's Park","Waterloo"],
"White City" : ["East Acton","Shepherd's Bush","Shepherd's Bush Market","Latimer Road"],
"Whitechapel" : ["Aldgate East","Shadwell","Shoreditch High Street","Stepney Green"],
"Willesden Green" : ["Dollis Hill","Kilburn"],
"Willesden Junction" : ["Harlesden","Kensal Green","Acton Central","Shepherd's Bush","Kensal Rise"],
"Wimbledon" : ["Wimbledon Park"],
"Wimbledon Park" : ["Southfields","Wimbledon"],
"Wood Green" : ["Bounds Green","Turnpike Lane"],
"Woodford" : ["Roding Valley","South Woodford"],
"Woodgrange Park" : ["Wanstead Park","Barking"],
"Woodside Park" : ["Totteridge & Whetstone","West Finchley"],
"Woolwich Arsenal" : ["King George V"]
}


In [4]:
max_len = 0
best_station = []
for key in list(tubemap.keys()):
    if len(tubemap[key]) >= max_len:
        max_len = len(tubemap[key])
        best_station = key
        
print(best_station,tubemap[best_station])

Bank & Monument ['Liverpool Street', "St. Paul's", 'Shadwell', 'London Bridge', 'Moorgate', 'Waterloo', 'Tower Hill', 'Cannon Street']


In [5]:
list(entry_exit_data['Station'])

['Upney',
 'Dagenham East',
 'Becontree',
 'Dagenham Heathway',
 'Barking',
 'Mill Hill East',
 'West Finchley',
 'Brent Cross',
 'Totteridge & Whetstone',
 'Woodside Park',
 'High Barnet',
 'Burnt Oak',
 'Edgware',
 'Colindale',
 'Finchley Central',
 'Hendon Central',
 'East Finchley',
 'Golders Green',
 'South Kenton',
 'North Wembley',
 'Kenton',
 'Sudbury Town',
 'Kensal Green',
 'Stonebridge Park',
 'Harlesden',
 'Alperton',
 'Kilburn Park',
 'Preston Road',
 'Neasden',
 'Dollis Hill',
 'Queensbury',
 'Willesden Junction',
 'Northwick Park',
 'Kingsbury',
 'Wembley Central',
 "Queen's Park",
 'Kilburn',
 'Willesden Green',
 'Wembley Park',
 'Brondesbury',
 'Brondesbury Park',
 'Kensal Rise',
 'Penge West',
 'Crystal Palace',
 'Anerley',
 'Hampstead',
 'Mornington Crescent',
 'Chalk Farm',
 'Belsize Park',
 'Swiss Cottage',
 'Kentish Town',
 'Goodge Street',
 'Finchley Road',
 'West Hampstead',
 'Russell Square',
 'Euston Square',
 'Camden Town',
 'Warren Street',
 'Holborn',
 'Eus

In [6]:
for station in list(tubemap.keys()):
    if station not in list(entry_exit_data['Station']):
        print(station)

In [7]:
for station in list(entry_exit_data['Station']):
    if station not in list(tubemap.keys()):
        print(station)

In [8]:
# Check bidirectional connections
errors = []
for station, connections in tubemap.items():
    for connected_station in connections:
        if connected_station not in tubemap:
            errors.append(f"{connected_station} listed in {station} does not exist in tubemap.")
        elif station not in tubemap[connected_station]:
            errors.append(f"Connection missing: {connected_station} to {station}")

# Check for isolated nodes
for station, connections in tubemap.items():
    if not connections:
        errors.append(f"{station} is isolated and has no connections.")

# Output errors
if errors:
    for error in errors:
        print(error)
else:
    print("No errors found. All connections are bidirectional and all stations are connected.")


No errors found. All connections are bidirectional and all stations are connected.


In [9]:
"""
#London Underground Journey Planner - www.101computing.net/london-underground-journey-planner/
def find_shortest_path(graph, start, end):
    if start==end:
        return [start]
    
    # Initialize the queue with the starting node and a path containing only the start
    queue = deque([(start, [start])])
    
    while queue:
        (vertex, path) = queue.popleft()
        
        # Check all adjacent nodes of the current vertex
        for next_node in graph[vertex]:
            if next_node in path:
                continue  # Skip nodes already in the path to avoid cycles
            
            # Construct the new path
            new_path = path + [next_node]
            
            # If we reach the end node, return the path
            if next_node == end:
                return new_path
            
            # Add the new path to the queue
            queue.append((next_node, new_path))
    
    return None  # Return None if no path is found       
        
def find_shortest_paths(graph, station_list, size):
    all_paths = np.empty((size, size), dtype=object)
    path_cache = {}  # Dictionary to store computed paths

    with tqdm(total=size**2, desc="Processing Paths") as pbar_outer:
        for i, stationFrom in enumerate(station_list[:size]):
            for j, stationTo in enumerate(station_list[:size]):
                # Check if path is already computed in the opposite direction
                if (stationTo, stationFrom) in path_cache:
                    path = path_cache[(stationTo, stationFrom)][::-1]  # Reverse the cached path
                else:
                    path = find_shortest_path(graph, stationFrom, stationTo)
                    path_cache[(stationFrom, stationTo)] = path  # Cache the path
                all_paths[i, j] = path
                pbar_outer.update(1)

    return all_paths
"""

'\n#London Underground Journey Planner - www.101computing.net/london-underground-journey-planner/\ndef find_shortest_path(graph, start, end):\n    if start==end:\n        return [start]\n    \n    # Initialize the queue with the starting node and a path containing only the start\n    queue = deque([(start, [start])])\n    \n    while queue:\n        (vertex, path) = queue.popleft()\n        \n        # Check all adjacent nodes of the current vertex\n        for next_node in graph[vertex]:\n            if next_node in path:\n                continue  # Skip nodes already in the path to avoid cycles\n            \n            # Construct the new path\n            new_path = path + [next_node]\n            \n            # If we reach the end node, return the path\n            if next_node == end:\n                return new_path\n            \n            # Add the new path to the queue\n            queue.append((next_node, new_path))\n    \n    return None  # Return None if no path is fo

In [12]:
find_shortest_path(tubemap,'Oval','White City')

['Oval',
 'Stockwell',
 'Vauxhall',
 'Pimlico',
 'Victoria',
 'Sloane Square',
 'South Kensington',
 'Gloucester Road',
 "Earl's Court",
 'Kensington (Olympia)',
 "Shepherd's Bush",
 'White City']

In [11]:
def find_shortest_path(graph, start, end, timeout=5):
    if start == end:
        return [start]
    
    queue = deque([(start, [start])])
    start_time = time.time()
    current_timeout = timeout
    
    while queue:
        vertex, path = queue.popleft()
        
        if time.time() - start_time > current_timeout:
            current_timeout += 10
            path = find_shortest_path(graph, end, start, timeout=current_timeout)
            if path:
                return path[::-1]
            return None
        
        for next_node in graph[vertex]:
            if next_node in path:
                continue
            
            new_path = path + [next_node]
            
            if next_node == end:
                return new_path
            
            queue.append((next_node, new_path))
    
    return None

def find_shortest_paths(graph, station_list, size):
    all_paths = np.empty((size, size), dtype=object)
    path_cache = {}
    
    save_folder = "Final saved rows"
    os.makedirs(save_folder, exist_ok=True)
    
    saved_rows = set()
    for filename in os.listdir(save_folder):
        if filename.startswith('saved_row_') and filename.endswith('.pkl'):
            try:
                row_index = int(filename.split('_')[2].split('.')[0]) - 1
                saved_rows.add(row_index)
            except ValueError:
                continue
    
    print("Saved rows:",saved_rows)

    with tqdm(total=size**2, desc="Processing Paths") as pbar_outer:
        for i, stationFrom in enumerate(station_list[:size]):
            print("From:",stationFrom,"\n")
            if i in saved_rows:
                file_path = os.path.join(save_folder, f'saved_row_{i+1}.pkl')
                with open(file_path, 'rb') as f:
                    all_paths[i, :] = pickle.load(f)
                pbar_outer.update(size)
                continue

            for j, stationTo in enumerate(station_list[:size]):
                if i == j:
                    all_paths[i, j] = [stationFrom]
                    pbar_outer.update(1)
                    continue

                if all_paths[j, i] is not None:
                    all_paths[i, j] = all_paths[j, i][::-1]
                else:
                    if (stationFrom, stationTo) in path_cache:
                        path = path_cache[(stationFrom, stationTo)]
                    else:
                        path = find_shortest_path(graph, stationFrom, stationTo)
                        if path is None:
                            path = find_shortest_path(graph, stationTo, stationFrom)
                            if path:
                                path = path[::-1]
                        path_cache[(stationFrom, stationTo)] = path
                    all_paths[i, j] = path
                
                pbar_outer.update(1)
            
            file_path = os.path.join(save_folder, f'saved_row_{i+1}.pkl')
            with open(file_path, 'wb') as f:
                pickle.dump(all_paths[i, :], f)
    
    return all_paths

In [ ]:
def overlap(path1, path2):
    shared_path = []
    
    min_length = min(len(path1), len(path2))
    
    for i in range(min_length):
        if path1[-1-i] == path2[-1-i]:
            shared_path.insert(0, path1[-1-i])
        else:
            break
    
    return shared_path

# overlap(path1,path2)

In [ ]:
path1 = find_shortest_path(tubemap,"Oval","London Bridge")
path1

In [ ]:
path2 = find_shortest_path(tubemap,"Woodford","Grange Hill")
path2

In [ ]:
station_list = list(entry_exit_data['Station'])
full = len(station_list)


size = full

all_paths = find_shortest_paths(tubemap, station_list, size)

save_folder = "Final saved rows"
file_path = os.path.join(save_folder, 'station_mapping.pkl')

# Export data to a file
with open(file_path, 'wb') as f:
    pickle.dump(all_paths, f)

In [ ]:
save_folder = "Final saved rows"
big_array = []

# Iterate over all files from saved_row_1.pkl to saved_row_333.pkl
for i in range(1, 333 + 1):
    file_path = os.path.join(save_folder, f'saved_row_{i}.pkl')
    
    with open(file_path, 'rb') as f:
        x = pickle.load(f)
    
    # Append the array to the big_array
    big_array.append(x)

# Convert big_array to a numpy array if needed
# big_array = np.array(big_array)

print("Big array created with length:", len(big_array))

In [ ]:
tube_paths = np.array(big_array,dtype=object)
tube_paths[116,1]

In [ ]:
np.array(list(tube_paths_tuples)).shape

In [ ]:
tube_paths

tube_paths_tuples = [[None for _ in range(len(row))] for row in tube_paths]

# convert to tuples for easy finding of trips
for index1, row in enumerate(tube_paths):
    for index2, column in enumerate(row):
        if len(column)==1:
            tube_paths_tuples[index1][index2] = [(column[0],column[0])]
        else:
            tube_paths_tuples[index1][index2] = [(column[i], column[i + 1]) for i in range(len(column) - 1)]
            
# Export data to a file
with open('tube_tuple_pairs.pkl', 'wb') as f:
    pickle.dump(tube_paths_tuples, f)

In [ ]:
subway_tuples[0]

In [ ]:
np.array(subway_tuples).shape

In [ ]:
with open('tube_tuple_pairs.pkl', 'rb') as f:
    subway_tuples = pickle.load(f)
    

subway_tuples[0]

In [ ]:
# Initialize the dictionary
pairs_dict = {}

# Iterate over tube_paths with a progress bar to populate pairs_dict
for row in tqdm(tube_paths, desc="Processing tube_paths rows"):
    for column in row:
        if len(column) == 1:
            key = (column[0], column[0])
            if key not in pairs_dict:
                pairs_dict[key] = []
        elif len(column) == 2:
            key = tuple(column)
            if key not in pairs_dict:
                pairs_dict[key] = []

# Load subway_tuples
with open('tube_tuple_pairs.pkl', 'rb') as f:
    subway_tuples = pickle.load(f)
    
print(subway_tuples)


# Get the list of stations
entry_exit_data = pd.read_excel('Expanded Station Entry Exit Data2.xls')
subway_station_list = list(entry_exit_data['Station'])

# Create a set for faster membership testing
station_set = set(subway_station_list)

# Flatten subway_tuples for faster access
flat_subway_tuples = {(i, j): set(subway_tuples[i][j]) for i in range(len(subway_tuples)) for j in range(len(subway_tuples[i]))}

# Total number of iterations for progress bar
total_iterations = len(subway_station_list) ** 2

# Iterate over stations with a progress bar
with tqdm(total=total_iterations, desc="Processing station pairs") as pbar:
    for origin in subway_station_list:
        for destination in subway_station_list:
            key = (origin, destination)
            if key in pairs_dict:
                for (i, j), path_set in flat_subway_tuples.items():
                    if key in path_set and subway_tuples[i][j][0][0] not in pairs_dict[key]:
                        pairs_dict[key].append([subway_tuples[i][j][0][0],subway_tuples[i][j][-1][1]])
            pbar.update(1)
            
with open('pairs dictionary.pkl', 'wb') as f:
        pickle.dump(pairs_dict, f)

In [ ]:
pairs_dict[('Woodford','Roding Valley')]